In [ ]:
# For tips on running notebooks in Google Colab:
# `pip install eegdash`
%matplotlib inline


# Challenge 2: Predicting the p-factor from EEG

This tutorial presents Challenge 2: regression of the p-factor (a general psychopathology factor) from EEG recordings.
The objective is to identify reproducible EEG biomarkers linked to mental health outcomes.

The challenge encourages learning physiologically meaningful signal representations.
Models of any size should emphasize robust, interpretable features that generalize across subjects,
sessions, and acquisition sites.

Unlike a standard in-distribution classification task, this regression problem stresses out-of-distribution robustness
and extrapolation. The goal is not only to minimize error on seen subjects, but also to transfer effectively to unseen data.

Ensure the dataset is available locally. If not, see the [dataset download guide](https://eeg2025.github.io/data/#downloading-the-data)

This tutorial is divided as follows:
1. **Loading the data**
2. **Wrap the data into a PyTorch-compatible dataset**
3. **Define, train and save a model**


## 1. Loading the data




In [ ]:
import random
from pathlib import Path
from eegdash import EEGChallengeDataset
from braindecode.preprocessing import create_fixed_length_windows
from braindecode.datasets.base import EEGWindowsDataset, BaseConcatDataset, BaseDataset

### 2. Define local path and download the data

In this challenge 2 example, we load the EEG 2025 release using EEG Dash and Braindecode,
we load all the public datasets available in the EEG 2025 release.



In [ ]:
# The first step is define the cache folder!
cache_dir = (Path.home() / "mne_data" / "eeg_challenge_cache").resolve()

# Creating the path if it does not exist
cache_dir.mkdir(parents=True, exist_ok=True)

# We define the list of releases to load.
# Here, all releases are loaded, i.e., 1 to 11.
release_list = ["R{}".format(i) for i in [5]]  # range(1, 11 + 1)]

# For this tutorial, we will only load the "resting state" recording,
# but you may use all available data.
all_datasets_list = [
    EEGChallengeDataset(
        release=release,
        query=dict(
            task="RestingState",
        ),
        description_fields=[
            "subject",
            "session",
            "run",
            "task",
            "age",
            "gender",
            "sex",
            "p_factor",
        ],
        cache_dir=cache_dir,
    )
    for release in release_list
]
print("Datasets loaded")

### Combine the datasets into single one
Here, we combine the datasets from the different releases into a single
``BaseConcatDataset`` object.



In [ ]:
all_datasets = BaseConcatDataset(all_datasets_list)
print(all_datasets.description)

### Inspect your data
We can check what is inside the dataset consuming the
MNE-object inside the Braindecode dataset.

The following snippet, if uncommented, will show the first 10 seconds of the raw EEG signal.
We can also inspect the data further by looking at the events and annotations.
We strong recommend you to take a look into the details and check how the events are structured.



In [ ]:
# raw = all_datasets.datasets[0].raw  # mne.io.Raw object
# print(raw.info)

# raw.plot(duration=10, scalings="auto", show=True)

# print(raw.annotations)

SFREQ = 100

## Wrap the data into a PyTorch-compatible dataset

The class below defines a dataset wrapper that will extract 2-second windows,
uniformly sampled over the whole signal. In addition, it will add useful information
about the extracted windows, such as the p-factor, the subject or the task.



In [ ]:
class DatasetWrapper(BaseDataset):
    def __init__(self, dataset: EEGWindowsDataset, crop_size_samples: int, seed=None):
        self.dataset = dataset
        self.crop_size_samples = crop_size_samples
        self.rng = random.Random(seed)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        X, _, crop_inds = self.dataset[index]

        # P-factor label:
        p_factor = self.dataset.description["p_factor"]
        p_factor = float(p_factor)

        # Additional information:
        infos = {
            "subject": self.dataset.description["subject"],
            "sex": self.dataset.description["sex"],
            "age": float(self.dataset.description["age"]),
            "task": self.dataset.description["task"],
            "session": self.dataset.description.get("session", None) or "",
            "run": self.dataset.description.get("run", None) or "",
        }

        # Randomly crop the signal to the desired length:
        i_window_in_trial, i_start, i_stop = crop_inds
        assert i_stop - i_start >= self.crop_size_samples, f"{i_stop=} {i_start=}"
        start_offset = self.rng.randint(0, i_stop - i_start - self.crop_size_samples)
        i_start = i_start + start_offset
        i_stop = i_start + self.crop_size_samples
        X = X[:, start_offset : start_offset + self.crop_size_samples]

        return X, p_factor, (i_window_in_trial, i_start, i_stop), infos


# Filter out recordings that are too short
all_datasets = BaseConcatDataset(
    [ds for ds in all_datasets.datasets if ds.raw.n_times >= 4 * SFREQ]
)

# Create 4-seconds windows with 2-seconds stride
windows_ds = create_fixed_length_windows(
    all_datasets,
    window_size_samples=4 * SFREQ,
    window_stride_samples=2 * SFREQ,
    drop_last_window=True,
)

# Wrap each sub-dataset in the windows_ds
windows_ds = BaseConcatDataset(
    [DatasetWrapper(ds, crop_size_samples=2 * SFREQ) for ds in windows_ds.datasets]
)

## Define, train and save a model
Now we have our pytorch dataset necessary for the training!

Below, we define a simple EEGNetv4 model from Braindecode and train it for one epoch
using pure PyTorch code.
However, you can use any pytorch model you want, or training framework.



In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import optim
from torch.nn.functional import l1_loss
from braindecode.models import EEGNetv4

# Use GPU if available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create PyTorch Dataloader
dataloader = DataLoader(windows_ds, batch_size=10, shuffle=True)

# Initialize model
model = EEGNetv4(n_chans=129, n_outputs=1, n_times=2 * SFREQ).to(DEVICE)

# All the braindecode models expect the input to be of shape (batch_size, n_channels, n_times)
# and have a test coverage about the behavior of the model.
print(model)

# Specify optimizer
optimizer = optim.Adamax(params=model.parameters(), lr=0.002)

# Train model for 1 epoch
for epoch in range(1):
    for idx, batch in enumerate(dataloader):
        # Reset gradients
        optimizer.zero_grad()

        # Unpack the batch
        X, y, crop_inds, infos = batch
        X = X.to(dtype=torch.float32, device=DEVICE)
        y = y.to(dtype=torch.float32, device=DEVICE).unsqueeze(1)

        # Forward pass
        y_pred = model(X)

        # Compute loss
        loss = l1_loss(y_pred, y)
        print(f"Epoch {0} - step {idx}, loss: {loss.item()}")

        # Gradient backpropagation
        loss.backward()
        optimizer.step()

# Finally, we can save the model for later use
torch.save(model.state_dict(), "./example_submission_challenge_2/weights.pt")